In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
import itertools
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
import copy
import scipy
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve


[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load Fasttext Unsupervised Model

In [2]:
model_2 = fasttext.load_model("models/ft_unsupervised_N_2.bin")
model_3 = fasttext.load_model("models/ft_unsupervised_N_3.bin")

### Make dataframes

In [3]:
# random sample
text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
text_labels = list(map(str, map(int, text_labels)))
text_TK = [CT.tokenize(x) for x in text]

df_dict = {'raw_comment': text,
           'tokenized_comment': text_TK, 'label': text_labels}
df_sample = pd.DataFrame(df_dict)
df_sample.to_csv('datasets/random_sample_data.csv', index=False)


### Remove Unnecessary Comments

In [4]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

In [5]:
df_sample.tail()


,raw_comment,tokenized_comment,label
4477,all the flags are sikh religious flags. they d...,all the flag be sikh religious flag they do no...,1
4478,Salute the farmers,salute the farmer,0
4479,Let's also remember how many farmers lost thei...,let be also remember how many farmer lose thei...,0
4480,ABP news you guys dont have heart 8 farmers go...,abp news you guy dont have heart 8 farmer get ...,0
4481,These new laws actually give the power to the ...,these new law actually give the power to the f...,1


In [6]:
df_sample['label'].value_counts()


0    1846
1    1408
Name: label, dtype: int64

### Train test split

In [7]:

X_train, X_test, y_train, y_test = train_test_split(df_sample['tokenized_comment'],
                                                    df_sample['label'], test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_sample['label'])


In [ ]:
print('X_train: ' ,len(X_train))
print('X_test: ' ,len(X_test))

### Classification

In [8]:
class FastTextTransformer(BaseEstimator, TransformerMixin):
    """ Convert texts into their mean fastText vectors """

    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.stack([np.mean([self.model[w] for w in text.split()], 0) for text in X])


def classify(small_model, predictor, lines, Y):
    classifier = make_pipeline(
        FastTextTransformer(model=small_model),
        predictor
    ).fit(
        lines,
        Y
    )
    return classifier


### Classifier

In [ ]:
# Logistic Regression
LR_Normal = classify(model_2, LogisticRegression(random_state=1), X_train, y_train)
# SVM
SVM_Normal = classify(model_2, svm.SVC(), X_train, y_train)
# KNN
KNN_Normal = classify(model_2, KNeighborsClassifier(n_neighbors=2), X_train, y_train)


# Logistic Regression
LR_Normal3 = classify(model_3, LogisticRegression(
    random_state=1), X_train, y_train)
# SVM
SVM_Normal3 = classify(model_3, svm.SVC(), X_train, y_train)
# KNN
KNN_Normal3 = classify(model_3, KNeighborsClassifier(
    n_neighbors=2), X_train, y_train)

models = []
models.append(('LR Normal N=2', LR_Normal))
models.append(('SVM Normal N=2', SVM_Normal))
models.append(('KNN Normal N=2', KNN_Normal))
models.append(('LR Normal N=3', LR_Normal3))
models.append(('SVM Normal N=3', SVM_Normal3))
models.append(('KNN Normal N=3', KNN_Normal3))


In [ ]:
file = open('results/output_normal.txt', 'w+')
file.close()

outfile = open("results/output_normal.txt", "a")
for i, v in models:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


### Active Learning

In [9]:
X_seed, X_expand, y_seed, y_expand = train_test_split(X_train,
                                                      y_train, test_size=0.99,
                                                      random_state=41,
                                                      stratify=y_train)


In [ ]:
print('X_seed: ',len(X_seed))
print('X_expand: ',len(X_expand))

In [ ]:
y_seed.value_counts()


In [ ]:
df_dict = {'seed_tokenized': X_seed,
           'label': y_seed}
df_seed = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/seed_data.csv', index=False)

df_dict = {'expansion_tokenized': X_expand,
           'label': y_expand}
df_expand = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/expand_data.csv', index=False)


### Cosine Similarity And Nearest Neighbors

In [10]:
def score(model, line, k):
    """ Returns a vector containing nearest neighbor scores w.r.t. all
        words in the model """
    # words contains all the words in the corpus
    lst1 = model.get_nearest_neighbors(line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)
    return v1


def NN(model: fasttext.FastText._FastText, line: str, K):
    """ Returns k fasttext nearest neighbors of a given string """
    return model.get_nearest_neighbors(line, k=K)


def get_NN(model: fasttext.FastText._FastText, lines: list, k: int):
    """ Returns k nearest neighbor scores of multiple strings"""
    scores = []
    for line in lines:
        scores.append(score(model, line, k))
    return scores

def cos_sim(a: np.array, b: np.array):
    """ Returns cosine similarity of two 1d arrays """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if(norm_a * norm_b == 0.0):
        return dot_product / (norm_a * norm_b + 0.001)
    return dot_product / (norm_a * norm_b)


def sim(x: np.array, y: np.array, sim_type: str):
    if(sim_type == 'cosine_sim'):
        return cos_sim(x, y)


def sim_matrix(A: np.array, B: np.array, sim_type: str):
    """ find similarity score matrix between A and B. 
        A,B: 2d matrix of embeddings/nearest neighbor scores.
        sim_type: String denoting type of similarity.
    """
    m, p = A.shape
    p, n = B.shape
    C = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(A[i, :], B[:, j], sim_type)
    return C


### Expansion Code (Random Sampling)

In [ ]:
def Expand_R(model: fasttext.FastText._FastText, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list, 
             expansion_set_labels: list, 
             batch_size: int, 
             k_neighbors: int, 
             random_rate: float):

    """
    Takes seed set and and expands the set using expansion_tokenised.
    Batch size: no. of texts to be inserted in one go
    k_neighbors: no. of neighbors for getting cosine similarity
    random_rate: fraction of amount taken for random sampling
    """
             
    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)
    cnt = int(random_rate * batch_size)
    
    expansion_predicted_labels = []
    expansion_true_labels = []
    confusion_matrices = []

    for i in range(1, len(M)):

        #print(M[i], end=' ')

        # select batchwise expansion text
        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # nearest neighbors
        seed_NN = get_NN(model, seed_TK, k_neighbors)
        exp_NN = get_NN(model, exp_TK, k_neighbors)

        A = np.array(seed_NN)
        B = np.array(exp_NN).T
        C = sim_matrix(A, B, "cosine_sim")

        # find rowwise (seed) index of highest similarity
        Y_ind = np.argmax(C, axis=0)
        # get labels
        Y = [seed_labels[x] for x in Y_ind]

        if(random_rate == 0.0):
            # no random sampling
            pass
        else:
            # random sampling
            Y_r = random.sample(range(0,len(Y)), cnt)
            for j in Y_r:
                Y[j] = exp_labels[j]

        # calc. expansion accuracy
        expansion_predicted_labels.extend(Y)
        expansion_true_labels.extend(exp_labels)
        confusion_matrices.append(metrics.confusion_matrix(exp_labels, Y))

        # expand seed set
        seed_labels.extend(Y)
        seed_TK.extend(exp_TK)

    return seed_TK, seed_labels, expansion_true_labels, expansion_predicted_labels, confusion_matrices


In [ ]:
print('model ', 'batch_size, ', 'Random Rate, ' 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

models = [('fasttext', 'models/ft_unsupervised_N_2.bin'),
          ('fasttext', 'models/ft_unsupervised_N_3.bin')]

batch_sizes = [10, 20]
random_rates = [0.1, 0.2]
outp_algorithms = ['knn']

big_confusion_r = []
big_results_r = []

for md in models:
    mdl = None
    if(md[0] == 'fasttext'):
        mdl = fasttext.load_model(md[1])
    
    for batch_size in batch_sizes:
        for random_rate in random_rates:
            seed_TK, seed_labels, _tk, _labels, confusion_matrices = Expand_R(model_2,
                                                                              list(
                                                                                  X_seed),
                                                                              list(
                                                                                  y_seed),
                                                                              list(
                                                                                  X_expand),
                                                                              list(
                                                                                  y_expand),
                                                                              batch_size,
                                                                              40,
                                                                              random_rate)
                                                                              
            for outp_algorithm in outp_algorithms:    

                alg2 = None
                if(outp_algorithm == 'lr'):
                    alg2 = LogisticRegression()
                elif(outp_algorithm == 'svm'):
                    alg2 = svm.SVC()
                elif(outp_algorithm == 'knn'):
                    alg2 = KNeighborsClassifier(n_neighbors=2)

                # classify
                c_model = classify(mdl, alg2, seed_TK, seed_labels)
                y_predict = c_model.predict(X_test)
                c_accuracy = metrics.accuracy_score(y_test, y_predict)
                c_prfsw = list(precision_recall_fscore_support(y_test, y_predict, average='weighted'))
                c_prfsmi = list(precision_recall_fscore_support(y_test, y_predict, average='micro'))

                l1 = [md[1], batch_size, random_rate, len(seed_TK), outp_algorithm, c_accuracy]
                l1.extend(c_prfsw)
                l1.extend(c_prfsmi)

                big_results_r.append(l1)
                print(md[1], ',', batch_size, ',', random_rate, ',', len(seed_TK), ',', outp_algorithm, ',', c_accuracy)
                big_confusion_r.append(confusion_matrices)



In [ ]:
res_df = pd.DataFrame(big_results_r)
res_df.to_csv("results/big_results_r.csv", index=False, header=False)

res_df = pd.DataFrame(big_confusion_r)
res_df.to_csv("results/big_confusion_r.csv", index=False, header=False)

### Uncertainty Sampling

In [11]:
def Expand_U(model: fasttext.FastText._FastText, 
             algorithm: object, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list,
             expansion_set_labels: list, 
             batch_size: int,
             countMax: int):
    """ Uncertainty sampling.
    Expand seed set using expansion_set based on lowest confidance scores.
    max_threshold: max. probability for uncertainty selection """

    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)
    confusion_matrices = []
    a_p_r_f = []
    seed_sizes = []

    # exp_TK_certain will be the list of comments having high proba score
    exp_TK_certain = []
    exp_TK_certain_labels = []


    for i in range(1, len(M)):

        #print(M[i], end=' ')

        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # take A as training and B as test and store probs in C
        small_model = classify(model, algorithm, seed_TK, seed_labels)
        # store classwise prob. scores
        C = small_model.predict_proba(exp_TK)
        # Uncertainty sampling scores
        C_abs_diff = [(abs(x[0] - x[1])) for x in C]

        # store accuracies
        confusion_matrices.append(metrics.confusion_matrix(y_test, small_model.predict(X_test)))

        y_predict = small_model.predict(X_test)
        c_accuracy = metrics.accuracy_score(y_test, y_predict)
        c_prfsw = list(precision_recall_fscore_support(
            y_test, y_predict, average='weighted'))
        c_prfsmi = list(precision_recall_fscore_support(
            y_test, y_predict, average='micro'))
        
        l1 = []
        l1.append(c_accuracy)
        l1.extend(c_prfsw[0:3])
        l1.extend(c_prfsmi[0:3])

        a_p_r_f.append(l1)
        seed_sizes.append(len(seed_TK))

        # Sort lists in ascending order of probabilities from C_abs_diff
        sorted_lists = sorted(zip(exp_labels, exp_TK, C, C_abs_diff), key=lambda x: x[3])
        exp_labels, exp_TK, C_sorted, score = [[x[i] for x in sorted_lists] for i in range(4)]

        Y_uncertain = []
        exp_TK_uncertain = []
        for j in range(len(C_sorted)):
            max_value = max(C_sorted[j])
            max_index = str(np.argmax(C_sorted[j]))

            # label the comments whose score is less than threshold
            if(j < countMax):

                exp_TK_uncertain.append(exp_TK[j])
                Y_uncertain.append(exp_labels[j])
            else:
                exp_TK_certain.append(exp_TK[j])
                exp_TK_certain_labels.append(exp_labels[j])

        # expand the seed set
        seed_labels.extend(Y_uncertain)
        seed_TK.extend(exp_TK_uncertain)


    return seed_TK, seed_labels, exp_TK_certain, exp_TK_certain_labels, confusion_matrices, a_p_r_f, seed_sizes


### Automatic 

In [13]:
print('model ', 'algorithm, ', 'batch_size, ', 'best_amount, ', 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

models = [('fasttext', 'models/ft_unsupervised_N_2.bin'),
          ('fasttext', 'models/ft_unsupervised_N_3.bin')]
algorithms = ['lr', 'svm', 'knn']
batch_sizes = [10, 20, 30, 40, 50, 100]
best_amounts = [1, 2, 5]
outp_algorithms = ['lr', 'svm', 'knn']

big_confusion = []
big_results = []
big_analysis = []
big_ss = []
for md in models:
    mdl = None
    if(md[0] == 'fasttext'):
        mdl = fasttext.load_model(md[1])

    for algorithm in algorithms:
        alg = None
        if(algorithm == 'lr'):
            alg = LogisticRegression()
        elif(algorithm == 'svm'):
            alg = svm.SVC(probability=True)
        elif(algorithm == 'knn'):
            alg = KNeighborsClassifier(n_neighbors=10)

        for batch_size in batch_sizes:
            for best_amount in best_amounts:

                X_uncertain_exp_u, y_uncertain_exp_u, X_certain_exp_u, y_certain_exp_u, confusion_matrices, a_p_r_f, ss = Expand_U(mdl,
                                                                                                                      alg,
                                                                                                                      list(
                                                                                                                          X_seed),
                                                                                                                      list(
                                                                                                                          y_seed),
                                                                                                                      list(
                                                                                                                          X_expand),
                                                                                                                      list(
                                                                                                                          y_expand),
                                                                                                                      batch_size,
                                                                                                                      best_amount
                                                                                                                      )
                for outp_algorithm in outp_algorithms:
                    alg2 = None
                    if(outp_algorithm == 'lr'):
                        alg2 = LogisticRegression()
                    elif(outp_algorithm == 'svm'):
                        alg2 = svm.SVC()
                    elif(outp_algorithm == 'knn'):
                        alg2 = KNeighborsClassifier(n_neighbors=2)

                    # classify
                    c_model = classify(mdl, alg2, X_uncertain_exp_u, y_uncertain_exp_u)
                    y_predict = c_model.predict(X_test)
                    c_accuracy = metrics.accuracy_score(y_test, y_predict)
                    c_prfsw = list(precision_recall_fscore_support(y_test, y_predict, average='weighted'))
                    c_prfsmi = list(precision_recall_fscore_support(y_test, y_predict, average='micro'))

                    l1 = [md[1], algorithm, batch_size, best_amount, len(
                        X_uncertain_exp_u), outp_algorithm, c_accuracy]
                    l1.extend(c_prfsw)
                    l1.extend(c_prfsmi)


                    big_results.append(l1)
                    print(md[1], ',', algorithm, ',', batch_size, ',', best_amount, ',', len(X_uncertain_exp_u), ',', outp_algorithm, ',', c_accuracy)
                    big_confusion.append(confusion_matrices)
                    big_analysis.append(a_p_r_f)
                    big_ss.append(ss)


model  algorithm,  batch_size,  best_amount,  expanded_size,  outp_algorithm,  c_accuracy


C:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


models/ft_unsupervised_N_2.bin , lr , 10 , 1 , 283 , lr , 0.7373271889400922
models/ft_unsupervised_N_2.bin , lr , 10 , 1 , 283 , svm , 0.7188940092165899
models/ft_unsupervised_N_2.bin , lr , 10 , 1 , 283 , knn , 0.6390168970814132


C:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


models/ft_unsupervised_N_2.bin , lr , 10 , 2 , 540 , lr , 0.7434715821812596
models/ft_unsupervised_N_2.bin , lr , 10 , 2 , 540 , svm , 0.7357910906298003
models/ft_unsupervised_N_2.bin , lr , 10 , 2 , 540 , knn , 0.6374807987711214
models/ft_unsupervised_N_2.bin , lr , 10 , 5 , 1311 , lr , 0.7603686635944701
models/ft_unsupervised_N_2.bin , lr , 10 , 5 , 1311 , svm , 0.7542242703533026
models/ft_unsupervised_N_2.bin , lr , 10 , 5 , 1311 , knn , 0.6682027649769585
models/ft_unsupervised_N_2.bin , lr , 20 , 1 , 154 , lr , 0.7465437788018433
models/ft_unsupervised_N_2.bin , lr , 20 , 1 , 154 , svm , 0.7004608294930875
models/ft_unsupervised_N_2.bin , lr , 20 , 1 , 154 , knn , 0.6221198156682027
models/ft_unsupervised_N_2.bin , lr , 20 , 2 , 282 , lr , 0.738863287250384
models/ft_unsupervised_N_2.bin , lr , 20 , 2 , 282 , svm , 0.7158218125960062
models/ft_unsupervised_N_2.bin , lr , 20 , 2 , 282 , knn , 0.6390168970814132
models/ft_unsupervised_N_2.bin , lr , 20 , 5 , 666 , lr , 0.723502

...

models/ft_unsupervised_N_3.bin , svm , 100 , 5 , 151 , lr , 0.7188940092165899
models/ft_unsupervised_N_3.bin , svm , 100 , 5 , 151 , svm , 0.7311827956989247
models/ft_unsupervised_N_3.bin , svm , 100 , 5 , 151 , knn , 0.6190476190476191
models/ft_unsupervised_N_3.bin , knn , 10 , 1 , 283 , lr , 0.7235023041474654
models/ft_unsupervised_N_3.bin , knn , 10 , 1 , 283 , svm , 0.65284178187404
models/ft_unsupervised_N_3.bin , knn , 10 , 1 , 283 , knn , 0.674347158218126
models/ft_unsupervised_N_3.bin , knn , 10 , 2 , 540 , lr , 0.7188940092165899
models/ft_unsupervised_N_3.bin , knn , 10 , 2 , 540 , svm , 0.7050691244239631
models/ft_unsupervised_N_3.bin , knn , 10 , 2 , 540 , knn , 0.6589861751152074
models/ft_unsupervised_N_3.bin , knn , 10 , 5 , 1311 , lr , 0.7511520737327189
models/ft_unsupervised_N_3.bin , knn , 10 , 5 , 1311 , svm , 0.7572964669738863
models/ft_unsupervised_N_3.bin , knn , 10 , 5 , 1311 , knn , 0.6758832565284179
models/ft_unsupervised_N_3.bin , knn , 20 , 1 , 154 ,

C:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


models/ft_unsupervised_N_3.bin , knn , 40 , 2 , 154 , svm , 0.5668202764976958
models/ft_unsupervised_N_3.bin , knn , 40 , 2 , 154 , knn , 0.6374807987711214
models/ft_unsupervised_N_3.bin , knn , 40 , 5 , 346 , lr , 0.7081413210445469
models/ft_unsupervised_N_3.bin , knn , 40 , 5 , 346 , svm , 0.6820276497695853
models/ft_unsupervised_N_3.bin , knn , 40 , 5 , 346 , knn , 0.6620583717357911
models/ft_unsupervised_N_3.bin , knn , 50 , 1 , 77 , lr , 0.6866359447004609
models/ft_unsupervised_N_3.bin , knn , 50 , 1 , 77 , svm , 0.6251920122887865
models/ft_unsupervised_N_3.bin , knn , 50 , 1 , 77 , knn , 0.554531490015361
models/ft_unsupervised_N_3.bin , knn , 50 , 2 , 128 , lr , 0.7342549923195084
models/ft_unsupervised_N_3.bin , knn , 50 , 2 , 128 , svm , 0.7188940092165899
models/ft_unsupervised_N_3.bin , knn , 50 , 2 , 128 , knn , 0.6451612903225806
models/ft_unsupervised_N_3.bin , knn , 50 , 5 , 281 , lr , 0.7327188940092166
models/ft_unsupervised_N_3.bin , knn , 50 , 5 , 281 , svm , 

C:\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


models/ft_unsupervised_N_3.bin , knn , 100 , 1 , 51 , svm , 0.5668202764976958
models/ft_unsupervised_N_3.bin , knn , 100 , 1 , 51 , knn , 0.5407066052227343
models/ft_unsupervised_N_3.bin , knn , 100 , 2 , 76 , lr , 0.7142857142857143
models/ft_unsupervised_N_3.bin , knn , 100 , 2 , 76 , svm , 0.6943164362519201
models/ft_unsupervised_N_3.bin , knn , 100 , 2 , 76 , knn , 0.5975422427035331
models/ft_unsupervised_N_3.bin , knn , 100 , 5 , 151 , lr , 0.6482334869431644
models/ft_unsupervised_N_3.bin , knn , 100 , 5 , 151 , svm , 0.5745007680491552
models/ft_unsupervised_N_3.bin , knn , 100 , 5 , 151 , knn , 0.6359447004608295


In [14]:
res_df = pd.DataFrame(big_results)
res_df.to_csv("results/big_results2.csv", index=False, header=False)


In [15]:
con_df = pd.DataFrame(big_confusion)
con_df.to_csv("results/big_confusion2.csv", index=False, header=False)


In [16]:
aprf_df = pd.DataFrame(big_analysis)
aprf_df.to_csv("results/big_analysis2.csv", index=False, header=False)


In [17]:
ss_df = pd.DataFrame(big_ss)
ss_df.to_csv("results/big_seed_sizes2.csv", index=False, header=False)
